# Overview

We’re building an iOS app that uses a Packet Tunnel VPN extension to capture all network traffic on the device, log useful metadata, and analyze it with a machine learning model to detect location-sharing services. The extension will handle traffic interception, buffering, and log rotation, while the host app manages VPN control, feature extraction, and Core ML inference. To make this work, we’ll cover project setup and entitlements, implementing the packet tunnel, designing buffers and file logging, and setting up communication between the extension and the app. We’ll also go through how to extract meaningful features from packet metadata and how to integrate an ML model for classification. Finally, we’ll discuss testing strategies and common pitfalls so the system works reliably without breaking device connectivity.


# Project Setup and Entitlements

### Setup

1) Create a new Xcode project
    * This is the Host Application -> It will run the VPN and ML model
2) Add a new target -> choose network extension -> select Packet Tunnel as the type
    * This will create an extension which can run separately from the app
    * There will be two bundles now, the app and the extension
3) Enable App Groups for both the app and the extension
    * Lets them share files
4) Enable Network Extensions
    * Signing and Capabilities -> add Network Extensions
    * Select Packet Tunnel Provider

### Entitlements

Extension:
* Network Extension Key
    * Packet Tunnel Provider
* Security Application Groups
    * My App’s Group
    
Application:
* Security Application Groups
    * My App’s Group


# Packet Tunnel Extension Basics

The Packet Tunnel extension (NEPacketTunnelProvider) is the engine that captures all device traffic. It acts like a VPN client, but instead of sending packets to a remote server, you can intercept them, log metadata, and then forward them back out.

### Core Lifecycle

1. startTunnel(options:completionHandler:)
    * Called when the system starts your VPN.
    * You configure network settings here (like IP and routing).
    * You also start your packet reading loop.
2. packetFlow.readPackets(...)
    * The main function that hands you raw packets.
    * You parse and log metadata here.
3. packetFlow.writePackets(...)
    * Must be called to send packets onward; otherwise, you’ll break connectivity.
4. stopTunnel(with:completionHandler:)
    * Cleanup when VPN stops.

In [ ]:
let ipv4 = NEIPv4Settings(addresses: ["10.0.0.2"], subnetMasks: ["255.255.255.0"])
ipv4.includedRoutes = [NEIPv4Route.default()]

Addresses: The Fake IP you give to the VPN interface

Inclulded Routes: NEIPv4Route.default() -> All traffic

In [ ]:
setTunnelNetworkSettings(settings) { error in ... }

This appies the settings and tells iOS to end every packet through the extension

In [ ]:
packetFlow.readPackets { packets, protocols in ... }
packetFlow.writePackets(packets, withProtocols: protocols)

This will read and do simple parsing of the packet stream

Then it will forward the packet

In [ ]:
startTunnel → configure fake IP + routes

loop:
    readPackets → parse + log → writePackets
    
stopTunnel → cleanup


# Time-Based Ring Buffer / In Memory Logging

The ring buffer is an in-memory queue that always keeps the last X seconds of packet metadata entries.

1. On every insert, we will push the current time and the packet
2. Periodically (or on every insert), we will remove the oldest packets
3. When the ML models runs, we will send the entire queue

Time Based is necessary due to the nature of the project. We will have a variable flow of packets and packets can appear inbetween location checks (even if it is unlikely).

This will be a custom datastructure using Swift Arrays. We can adjust to Linked Lists or Circular buffers if speed is necessary

# On Disk Storage

The in-memory buffer is only for short term storage.

Since the packet tunnel extension is sandboxed seperately from the app, we need to make an App Group Container.

In [ ]:
group.com.company.AppName

FileManager.default.containerURL(forSecurityApplicationGroupIdentifier: "group.com.company.AppName")

let logFile = url.appendingPathComponent("packets.log")
try data.append(to: logFile)



The App Group identifier will look like the above, which is followed by how each process can access the data.

The extension will append ML data, which can include timestamps, and packets involved.
The host app can open the files later for debugging or showing to the user.

The final two lines are used to append log data to a file.

# App Extension Communication

Communication will happen in 3 different ways:
1) via App Groups
2) via NEAppMessage
3) via UserNotifications

App Groups is used for persistant storage as discussed in the "On Disk Storage" section.

NEAppMessage will be used for short real time messages, like setting changes or opening/closing the VPN.

User Notifcations will push alerts even if the app is closed, which can be used for real time user notifications.

In [ ]:
// In host app
let manager = NETunnelProviderManager.shared()
let message: [String: Any] = ["window": 60, "sensitivity": "high"]
let data = try! JSONSerialization.data(withJSONObject: message)

manager.connection as? NETunnelProviderSession?.sendProviderMessage(data) { response in
    if let resp = response {
        print("Extension acknowledged: \(resp)")
    }
}

In [ ]:
// In PacketTunnelProvider
override func handleAppMessage(_ messageData: Data,
                               completionHandler: ((Data?) -> Void)?) {
    if let dict = try? JSONSerialization.jsonObject(with: messageData) as? [String: Any] {
        print("Received settings: \(dict)")
        // Update internal config here
    }
    completionHandler?("OK".data(using: .utf8))
}

The previous two sections of code is how to manage App Messages. This is a lightweight example which will need more development.

The next two sections of code is for User Notifications. First it requests permission and then how to send a notification. Again, more development is needed.

In [ ]:
// Request Permission
import UserNotifications

UNUserNotificationCenter.current().requestAuthorization(
    options: [.alert, .sound, .badge]
) { granted, error in
    print("Notifications granted: \(granted)")
}

In [ ]:
//Send Notification
import UserNotifications

let content = UNMutableNotificationContent()
content.title = "Network Alert"
content.body = "Potential location-sharing detected"
content.sound = .default

let request = UNNotificationRequest(identifier: UUID().uuidString,
                                    content: content,
                                    trigger: nil)

UNUserNotificationCenter.current().add(request) { error in
    if let error = error {
        print("Notification error: \(error)")
    }
}

# Host app: VPN session management

This part will consist of 3 core components:
1) NETunnelProviderManager
2) NETunnelProviderSession
3) Persistence 

Workflow:
1) Load or create the manager
2) Set protocol and provider
3) Save to preferences
4) Starting and stopping

The following is a simple example of how to implement this.

In [ ]:
// Load or create VPN manager
NETunnelProviderManager.loadAllFromPreferences { managers, error in
    let manager = managers?.first ?? NETunnelProviderManager()

    let proto = NETunnelProviderProtocol()
    proto.providerBundleIdentifier = "com.yourcompany.YourApp.PacketTunnel"
    proto.serverAddress = "127.0.0.1" // dummy, required

    manager.protocolConfiguration = proto
    manager.localizedDescription = "Traffic Monitor VPN"

    manager.isEnabled = true

    // Save to preferences
    manager.saveToPreferences { error in
        if error == nil {
            print("VPN config saved")
        }
    }
}

// Start VPN
manager.connection as? NETunnelProviderSession?.startVPNTunnel()

// Stop VPN
manager.connection as? NETunnelProviderSession?.stopVPNTunnel()

The first time, the system will prompt the user to allow the configurations.

Another thing to consider is only one VPN extension can run at a time on iOS.

The Host App will have the VPN controller, while the extension will have the VPN engine

# Running the ML model

First we need to import the ML model. In python, a library called "coremltools" is needed to convert the model to a model that iOS can use.

Next we need to load and run the model within the extension. This is necessary for real time predictions. The following is example code of how to do that.

In [ ]:
// Load model
let model = try? MyTrafficModel(configuration: MLModelConfiguration())

// Make prediction (input depends on model's expected format)
if let output = try? model.prediction(input: inputData) {
    if output.label == "location_sharing" {
        triggerNotification("Possible location-sharing detected")
    }
}

This will work in unison with the UserNotification library, to send notifications on a successful identification of location sharing packets.

Finally, we will need to develop a ML model context trigger, which will send the current bag of packets to the model. This is necessary so we do not run the model continously, only when the necessary requirements are met.

# UI (SwiftUI)

We will need several different "views". Views are different app screens.
1) Status View
2) Detection Log (History) View
3) Settings View

We will navigate this with a tab based layout. The following is a basic example of this.

In [ ]:
import SwiftUI

struct ContentView: View {
    var body: some View {
        TabView {
            StatusView()
                .tabItem {
                    Label("Status", systemImage: "network")
                }
            
            LogView()
                .tabItem {
                    Label("Logs", systemImage: "list.bullet")
                }
            
            SettingsView()
                .tabItem {
                    Label("Settings", systemImage: "gear")
                }
        }
    }
}

// Placeholder views for each tab
struct StatusView: View {
    var body: some View {
        VStack {
            Text("VPN Status: Running")
        }
        .padding()
    }
}

struct LogView: View {
    var body: some View {
        VStack {
            Text("Recent Detection Logs")
        }
        .padding()
    }
}

struct SettingsView: View {
    var body: some View {
        VStack {
            Text("Settings")
        }
        .padding()
    }
}

// Preview for Xcode
struct ContentView_Previews: PreviewProvider {
    static var previews: some View {
        ContentView()
    }
}

# Trouble Shooting

A real iOS device is needed to test networking code. While plugged in, you can build to target device, which will create an example app the hosts the app and the extension.

To test the extension, you would need to:
1) Connect the device to the mac via USB or USB-C
2) Trust the developer profile
    * Settings -> General -> Device Management -> Trust your development certificate.
3) Ensure all extensions are set up on in the Xcode project
4) Select device as target for the build
5) Run from Xcode
6) VPN will ask for authorization the first time
7) Rebuild and reinstall when extension logic is changed

For testing other features, we can use the similulator. This will be useful for UI mainly.

## Disclaimer

Do not trust the code provided too much. This was either pulled from websites or basic adaptations of it without testing.

I made this file because I currently cannot code on a Mac device, and this file should help speed things along.